In [12]:
import numpy as np
import tensorflow as tf

## Anchor Generation Layer

In [76]:
def _extract_cor(anchor):
    """
    Return width, height, x center, and y center for an anchor (window).
    """
    # anchor is 1X4
    w = anchor[2] - anchor[0] + 1
    h = anchor[3] - anchor[1] + 1
    x_ctr = anchor[0] + 0.5 * (w - 1)
    y_ctr = anchor[1] + 0.5*(w-1)
    return w,h,x_ctr,y_ctr

In [77]:
def _make_anchors(ws,hs,x_ctr,y_ctr):
    """
    Given a vector of widths (ws) and heights (hs) around a center (x_ctr,y_ctr),
    Return a set of anchor boxes in (w, h, x_ctr, y_ctr) format.
    """
    # ws,hs,x_ctr,y_ctr are numpy arrays
    w = ws[:,np.newaxis] # [1,4] -> [4,1] i.e it generate array of one more dimension
    # print('w in mkanchors: ',w)
    h = hs[:,np.newaxis] # [1,4] -> [4,1]
    # print('h in mkanchors: ',h)
    anchors = np.hstack((x_ctr - 0.5*(w-1),
                                            y_ctr - 0.5*(h-1),
                                            x_ctr + 0.5*(w-1),
                                            y_ctr + 0.5*(h-1))) # horizontal stack
    # print('anchors in mkanchors: ',anchors)
    return anchors

In [78]:
def _ratio_enum(anchor,ratios):
    """
    Enumerate a set of anchors for each aspect ratio wrt an anchor.
    """
    
    w,h,x_ctr,y_ctr = _extract_cor(anchor)
    # print(w,h,x_ctr,y_ctr)
    size = w*h
    # print('size='+str(size))
    size_ratios = size/ratios
    # print('size_ratios='+str(size_ratios))
    ws = np.round(np.sqrt(size_ratios))
    # print('ws='+str(ws))
    hs = np.round(ws*ratios)
    # print('hs='+str(hs))
    anchors = _make_anchors(ws,hs,x_ctr,y_ctr)
    # print('anchors in ratio_enu='+str(anchors))
    return anchors

In [79]:
def _scale_enum(anchor,scales):
    """
    Enumerate a set of anchors for each scale wrt an anchor.
    """
    w, h, x_ctr, y_ctr = _extract_cor(anchor)
    # print("anchor in scle_enum="+str(anchor))
    ws = w*scales
    hs = h*scales
    anchors = _make_anchors(ws,hs,x_ctr,y_ctr)
    # print("Final anchors in scale_enum="+str(anchors))
    return anchors

In [80]:
#  It generates 9 anchor boxes from a base anchor box

def generate_anchors(base_size=16,ratios=[0.5,1,2],scales=np.array([8,16,32])):
    base_anchor = np.array([1,1,base_size,base_size]) - 1
    # print(base_anchor)
    ratio_anchors = _ratio_enum(base_anchor,ratios)
    # print(ratio_anchors.shape)
    anchors_list=[]
    for i in range(ratio_anchors.shape[0]):
        anc = _scale_enum(ratio_anchors[i,:],scales)
        anchors_list.append(anc)
    anchors = np.vstack(anchors_list)
    return anchors

In [81]:
generate_anchors()

array([[ -84. ,  -34.5,   99. ,   60.5],
       [-176. ,  -82.5,  191. ,  108.5],
       [-360. , -178.5,  375. ,  204.5],
       [ -56. ,  -56. ,   71. ,   71. ],
       [-120. , -120. ,  135. ,  135. ],
       [-248. , -248. ,  263. ,  263. ],
       [ -36. ,  -85.5,   51. ,   89.5],
       [ -80. , -173.5,   95. ,  177.5],
       [-168. , -349.5,  183. ,  353.5]])

In [82]:
# create uniformly spaced grid with spacing equal to stride

def generate_anchors_pre_tf(height, width, feat_stride=16, anchor_scales=(8,16,32), anchor_ratios=(0.5,1,2)):
    """
    A wrapper function to generate anchors given different scales and
    ratios.
    """

    shift_x = tf.range(width) * feat_stride # [0,16,32,48] width
    shift_y = tf.range(height) * feat_stride # [0,16,32,48] height
    shift_x, shift_y = tf.meshgrid(shift_x, shift_y) # meshgrid cols, rows , meshgrid generates a grid of points in ND space
    # meshgrid enumerate shift_x row wise and shift_y col wise
    shift_x = tf.reshape(shift_x, shape=(-1,)) # reshape to 1D
    shift_y = tf.reshape(shift_y, shape=(-1,))
    shifts = tf.stack((shift_x, shift_y, shift_x, shift_y), axis=1) # vertical stack by row
    K = tf.multiply(width, height)
    shifts = tf.transpose(tf.reshape(shifts,shape=[1,K,4]),perm=[1,0,2]) # reshaping into Kx1x4

    anchors = generate_anchors(ratios=np.array(anchor_ratios), scales=np.array(anchor_scales)) # basic 9 anchor boxes of shape (9,4)
    A = anchors.shape[0] # 9
    anchor_constants = tf.constant(anchors.reshape((1, A, 4)), dtype=tf.int32) # reshape to 1x9x4
    
    length = K*A
    anchors_tf = tf.reshape(tf.add(anchor_constants,shifts),shape=[length,4]) # add shift to anchors element wise
    return tf.cast(anchors_tf,tf.float32),length

In [83]:
tensor_anchors, length = generate_anchors_pre_tf(height=600//16,width=800//16)
print("type of tensor_anchors: ",type(tensor_anchors))
print("tensor_anchors shape",tensor_anchors.shape)
print("length="+str(length))

type of tensor_anchors:  <class 'tensorflow.python.framework.ops.EagerTensor'>
tensor_anchors shape (16650, 4)
length=tf.Tensor(16650, shape=(), dtype=int32)


## For Bounding box regression coefficients

In [1]:
import numpy as np
import tensorflow as tf

In [7]:
# calculating bounding box regression coefficients
def bbox_transform(original_rois,gt_rois):
    original_widths = original_rois[:,2] - original_rois[:,0] + 1.0
    original_heights = original_rois[:,3] - original_rois[:,1] + 1.0
    original_ctr_x = original_rois[:,0] + 0.5 * original_widths
    original_ctr_y = original_rois[:,1] + 0.5 * original_heights

    gt_widths = gt_rois[:,2] - gt_rois[:,0] + 1.0
    gt_heights = gt_rois[:,3] - gt_rois[:,1] + 1.0
    gt_ctr_x = gt_rois[:,0] + 0.5 * gt_widths
    gt_ctr_y = gt_rois[:,1] + 0.5 * gt_heights

    targets_dx = (gt_ctr_x - original_ctr_x) / original_widths
    targets_dy = (gt_ctr_y - original_ctr_y) / original_heights
    targets_dw = np.log(gt_widths / original_widths)
    targets_dh = np.log(gt_heights / original_heights)

    targets = np.vstack((targets_dx, targets_dy, targets_dw, targets_dh)).transpose()
    return targets

In [9]:
bbox_transform(np.array([[-1,-1,15,15],[-1,-1,31,31]]),np.array([[0,0,16,16],[0,0,32,32]]))

array([[0.05882353, 0.05882353, 0.        , 0.        ],
       [0.03030303, 0.03030303, 0.        , 0.        ]])